# 含并⾏连结的⽹络
在2014年的ImageNet图像识别挑战赛中，⼀个名叫GoogLeNet的⽹络结构⼤放异彩 [1]。它虽然
在名字上向LeNet致敬，但在⽹络结构上已经很难看到LeNet的影⼦。 GoogLeNet吸收了NiN中⽹
络串联⽹络的思想，并在此基础上做了很⼤改进。在随后的⼏年⾥，研究⼈员对GoogLeNet进⾏
了数次改进，本节将介绍这个模型系列的第⼀个版本。
## Inception块
GoogLeNet中的基础卷积块叫作Inception块，得名于同名电影《盗梦空间》（Inception）。与上
⼀节介绍的NiN块相⽐，这个基础块在结构上更加复杂，如图5.8所⽰

![Inception块的结构](../img/inception.svg)

由图5.8可以看出， Inception块⾥有4条并⾏的线路。前3条线路使⽤窗口⼤小分别是1 × 1、 3 ×
3和5 × 5的卷积层来抽取不同空间尺⼨下的信息，其中中间2个线路会对输⼊先做1 × 1卷积来减
少输⼊通道数，以降低模型复杂度。第四条线路则使⽤3 × 3最⼤池化层，后接1 × 1卷积层来改变
通道数。 4条线路都使⽤了合适的填充来使输⼊与输出的⾼和宽⼀致。最后我们将每条线路的输
出在通道维上连结，并输⼊接下来的层中去。

Inception块中可以⾃定义的超参数是每个层的输出通道数，我们以此来控制模型复杂度。

In [1]:
import d2lzh as d2l
from mxnet import gluon, init, nd
from mxnet.gluon import nn

class Inception(nn.Block):
	# c1 - c4为每条线路里的层的输出通道数
	def __init__(self, c1, c2, c3, c4, **kwargs):
		super(Inception, self).__init__(**kwargs)
		# 线路1，单1 x 1卷积层
		self.p1_1 = nn.Conv2D(c1, kernel_size=1, activation='relu')
		# 线路2，1 x 1卷积层后接3 x 3卷积层
		self.p2_1 = nn.Conv2D(c2[0], kernel_size=1, activation='relu')
		self.p2_2 = nn.Conv2D(c2[1], kernel_size=3, padding=1, activation='relu')
		# 线路3，1 x 1卷积层后接5 x 5卷积层
		self.p3_1 = nn.Conv2D(c3[0], kernel_size=1, activation='relu')
		self.p3_2 = nn.Conv2D(c3[1], kernel_size=5, padding=2, activation='relu')
		# 线路4，3 x 3最大池化层后接1 x 1卷积层
		self.p4_1 = nn.MaxPool2D(pool_size=3, strides=1, padding=1)
		self.p4_2 = nn.Conv2D(c4, kernel_size=1, activation='relu')

	def forward(self, x):
		p1 = self.p1_1(x)
		p2 = self.p2_2(self.p2_1(x))
		p3 = self.p3_2(self.p3_1(x))
		p4 = self.p4_2(self.p4_1(x))
		return nd.concat(p1, p2, p3, p4, dim=1)  # 在通道维上连结输出

## GoogLeNet模型
GoogLeNet跟VGG⼀样，在主体卷积部分中使⽤5个模块（block），每个模块之间使⽤步幅为2的3×
3最⼤池化层来减小输出⾼宽。第⼀模块使⽤⼀个64通道的7 × 7卷积层。

In [2]:
b1 = nn.Sequential()
b1.add(nn.Conv2D(64,kernel_size=7,strides=2,padding=3,activation='relu'),
       nn.MaxPool2D(pool_size=3,strides=2,padding=1))

第⼆模块使⽤2个卷积层：⾸先是64通道的1 × 1卷积层，然后是将通道增⼤3倍的3 × 3卷积层。它
对应Inception块中的第⼆条线路。

In [3]:
b2 = nn.Sequential()
b2.add(nn.Conv2D(64, kernel_size=1, activation='relu'),
       nn.Conv2D(192, kernel_size=3, padding=1, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

第三模块串联2个完整的Inception块。第⼀个Inception块的输出通道数为64+128+32+32 = 256，
其中4条线路的输出通道数⽐例为64 : 128 : 32 : 32 = 2 : 4 : 1 : 1。其中第⼆、第三条线路先分别将
输⼊通道数减小⾄96/192 = 1/2和16/192 = 1/12后，再接上第⼆层卷积层。第⼆个Inception块
输出通道数增⾄128 + 192 + 96 + 64 = 480，每条线路的输出通道数之⽐为128 : 192 : 96 : 64 =
4 : 6 : 3 : 2。其中第⼆、第三条线路先分别将输⼊通道数减小⾄128/256 = 1/2和32/256 = 1/8。

In [4]:
b3 = nn.Sequential()
b3.add(Inception(64, (96, 128), (16, 32), 32),
       Inception(128, (128, 192), (32, 96), 64),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

第四模块更加复杂。它串联了5个Inception块，其输出通道数分别是192 + 208 + 48 + 64 = 512、
160+224+64+64 = 512、128+256+64+64 = 512、112+288+64+64 = 528和256+320+128+128 =
832。这些线路的通道数分配和第三模块中的类似，⾸先含3 × 3卷积层的第⼆条线路输出最多通
道，其次是仅含1×1卷积层的第⼀条线路，之后是含5×5卷积层的第三条线路和含3×3最⼤池化
层的第四条线路。其中第⼆、第三条线路都会先按⽐例减小通道数。这些⽐例在各个Inception块
中都略有不同。

In [5]:
b4 = nn.Sequential()
b4.add(Inception(192, (96, 208), (16, 48), 64),
       Inception(160, (112, 224), (24, 64), 64),
       Inception(128, (128, 256), (24, 64), 64),
       Inception(112, (144, 288), (32, 64), 64),
       Inception(256, (160, 320), (32, 128), 128),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

第五模块有输出通道数为256 + 320 + 128 + 128 = 832和384 + 384 + 128 + 128 = 1024的两
个Inception块。其中每条线路的通道数的分配思路和第三、第四模块中的⼀致，只是在具体数值
上有所不同。需要注意的是，第五模块的后⾯紧跟输出层，该模块同NiN⼀样使⽤全局平均池化
层来将每个通道的⾼和宽变成1。最后我们将输出变成⼆维数组后接上⼀个输出个数为标签类别
数的全连接层。

In [6]:
b5 = nn.Sequential()
b5.add(Inception(256, (160, 320), (32, 128), 128), 
       Inception(384, (192, 384), (48, 128), 128),
       nn.GlobalAvgPool2D())
net = nn.Sequential()
net.add(b1, b2, b3, b4, b5, nn.Dense(10))

GoogLeNet模型的计算复杂，而且不如VGG那样便于修改通道数。本节⾥我们将输⼊的⾼和宽
从224降到96来简化计算。下⾯演⽰各个模块之间的输出的形状变化。

In [7]:
X = nd.random.uniform(shape=(1, 1, 96, 96))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

sequential0 output shape:	 (1, 64, 24, 24)
sequential1 output shape:	 (1, 192, 12, 12)
sequential2 output shape:	 (1, 480, 6, 6)
sequential3 output shape:	 (1, 832, 3, 3)
sequential4 output shape:	 (1, 1024, 1, 1)
dense0 output shape:	 (1, 10)


## 获取数据和训练模型
我们使⽤⾼和宽均为96像素的图像来训练GoogLeNet模型。训练使⽤的图像依然来⾃FashionMNIST数据集。

In [8]:
lr, num_epochs, batch_size, ctx = 0.1, 5, 128, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx,num_epochs)

## ⼩结
- Inception块相当于⼀个有4条线路的⼦⽹络。它通过不同窗口形状的卷积层和最⼤池化层
来并⾏抽取信息，并使⽤1 × 1卷积层减少通道数从而降低模型复杂度。
- GoogLeNet将多个设计精细的Inception块和其他层串联起来。其中Inception块的通道数分
配之⽐是在ImageNet数据集上通过⼤量的实验得来的。
- GoogLeNet和它的后继者们⼀度是ImageNet上最⾼效的模型之⼀：在类似的测试精度下，
它们的计算复杂度往往更低。